In [1]:
import sqlite3

db = sqlite3.connect("WedgePart3.db")

In [2]:
cur = db.cursor()

In [3]:
db.commit()

In [4]:
import random
import os
from google.cloud import bigquery
from google.oauth2 import service_account

service_path = "C:\\Users\\mr_me\\Desktop\\ada-wedge\\"
service_file = 'wedge-project-ada-amh-c5fc54f5c97c.json' # this is your authentication information  
gbq_proj_id = 'wedge-project-ada-amh'  # change this to your project_id
gbp_dataset_id = 'Wedge_Data_Complete'
# And this should stay the same. 

 
private_key =service_path + service_file

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
#query for first table

query = ('''
SELECT
(EXTRACT(date FROM datetime)) AS date,
(EXTRACT(hour FROM datetime)) AS hour,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) as Transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
FROM `wedge-project-ada-amh.wedge_data.Wedge_Data_Complete`
WHERE
  card_no != 3
  AND department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY date, hour
ORDER BY date, hour
''')

query_job = client.query(
    query,
    location = 'US',
)

table_one = client.query(query)



with open('SalesDate.txt', 'w')as output_file:
    for line in table_one:
        row = [str(item) for item in line]
        output_file.write(','.join(row)+ '\n')

In [6]:
#create SalesDate in db
input_file = 'SalesDate.txt'
db = sqlite3.connect("AHendricks_Wedge_Pt3.db")
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS SalesDate''')
cur.execute('''CREATE TABLE SalesDate (
    date TIMESTAMP,
    hour TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

In [7]:
#insert query 1 data into SalesDate
with open(input_file, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO SalesDate (date, hour, sales, transactions, items)
        VALUES (?,?,?,?,?)''', line)
db.commit()

In [8]:
#query for second table

query = ('''
SELECT
card_no,
(EXTRACT(year FROM datetime)) AS year,
(EXTRACT(month FROM datetime)) AS month,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) as Transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items,

FROM `wedge-project-ada-amh.wedge_data.Wedge_Data_Complete`
WHERE
  card_no != 3
  AND department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY card_no, year, month
ORDER BY year, month
''')

query_job = client.query(
    query,
    location = 'US',
)

table_two = client.query(query)



with open('SalesOwner.txt', 'w')as output_file:
    for line in table_two:
        row = [str(item) for item in line]
        output_file.write(','.join(row)+ '\n')

In [9]:
#create SalesOwner in db
input_file = 'SalesOwner.txt'
db = sqlite3.connect("AHendricks_Wedge_Pt3.db")
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS SalesOwner''')
cur.execute('''CREATE TABLE SalesOwner (
    card_no INTEGER,
    year TIMESTAMP,
    month TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

In [10]:
#insert query 2 data into SalesOwner
with open(input_file, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO SalesOwner (card_no, year, month, sales, transactions, items)
        VALUES (?,?,?,?,?,?)''', line)
db.commit()

In [11]:
#query for third table

query = ('''
SELECT
upc,
description,
a.department,
b.dept_name,
  
(EXTRACT(year FROM datetime)) AS year,
(EXTRACT(month FROM datetime)) AS month,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) as Transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items,

FROM `wedge-project-ada-amh.wedge_data.Wedge_Data_Complete`a
LEFT JOIN `wedge-project-ada-amh.wedge_data.Dept_Name`b
ON a.department = b.department
WHERE
  card_no != 3
  AND a.department != 0
  AND a.department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY upc, description, department, b.dept_name,year, month
ORDER BY year, month
''')

query_job = client.query(
    query,
    location = 'US',
)

table_two = client.query(query)



with open('SalesProduct.txt', 'w')as output_file:
    for line in table_two:
        row = [str(item) for item in line]
        output_file.write(','.join(row)+ '\n')

In [12]:
#create SalesProduct in db
input_file = 'SalesProduct.txt'
db = sqlite3.connect("AHendricks_Wedge_Pt3.db")
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS SalesProduct''')
cur.execute('''CREATE TABLE SalesProduct (
    upc STRING,
    description STRING,
    department FLOAT,
    department_name STRING,
    year TIMESTAMP,
    month TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

In [13]:
#insert query 3 data into SalesProduct
with open(input_file, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO SalesProduct (upc, description, department, department_name, year, month, sales, transactions, items)
        VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()